In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("C:/train.csv", nrows = 100000)

In [ ]:
data.head()

In [ ]:
data["pickup_datetime"] = data["pickup_datetime"].str.replace(" UTC","")
data["pickup_datetime"] = pd.to_datetime(data["pickup_datetime"],format="%Y-%m-%d %H:%M:%S")

In [ ]:
data["year"]  = pd.DatetimeIndex(data["pickup_datetime"]).year
#extract month
data["month"] = pd.DatetimeIndex(data["pickup_datetime"]).month
data["month_name"] = data["month"].map({1:"JAN",2:"FEB",3:"MAR",
                                                4:"APR",5:"MAY",6:"JUN",
                                                7:"JUL",8:"AUG",9:"SEP",
                                                10:"OCT",11:"NOV",12:"DEC"
                                               })
#merge year month
data["month_year"] = data["year"].astype(str) + " - " + data["month_name"]
#extract week day 
data["week_day"]   = data["pickup_datetime"].dt.weekday_name
#extract day 
data["day"]        = data["pickup_datetime"].dt.day
#extract hour
data["hour"]        = data["pickup_datetime"].dt.hour 
data = data.sort_values(by = "pickup_datetime",ascending = False)

In [ ]:
total_fare = (data.groupby(["year","month","month_name",
                                     "month_year"])["fare_amount"].sum().reset_index())

In [ ]:
ts_fare = total_fare.copy()

In [ ]:
ts_fare["date"] = ts_fare["year"].astype(str) + "-" + ts_fare["month"].astype(str)
#selecting columns
ts_fare = ts_fare[["date","fare_amount"]]
#convert to date format
ts_fare["date"] = pd.to_datetime(ts_fare["date"],format = "%Y-%m")
ts_fare.index   = ts_fare["date"]
ts_fare = ts_fare.drop(columns  = ["date"],axis = 1)
ts_fare.head(10)

In [ ]:
log_ts_fare = np.log(ts_fare)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

#ARIMA model
def arima_model(time_series,p,d,q) :
    arima_model   = ARIMA(time_series , order = (p,d,q))
    results_arima = arima_model.fit(disp = -1)
    fitted_values = results_arima.fittedvalues
    print (results_arima.summary())

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
arima_model(log_ts_fare,0,1,2)